In [1]:
import numpy as np
from matplotlib import pyplot as plt 

## Iterated Best Response for finding Equilibria
In the setting of a 2 player game we wish to calculate the best response to a fixed strategy and iterate on it. Hopefully by doing so we will uncover the Nash Equilibrium for this particular game.

### Mathematics behind best response
So far as we have 4 categories that we determine they are superior in order Q1 > Q2 > Q3 > Q4. So we have to do our expectation going down in the game.

Let there be n applicants, broken into 4 categories, with d1, d2, d3, d4 size respectively.
f(mean, high) = frequency of high conventional metric talent (d1,d2) = (d1 + d2) / n
f(std, high) = frequency of high unconventional metric talent = d1/(d1+d2) = d3/(d3+d4)
f(mean, low) = 1-f(mean, high)
f(std, low) = 1-f(std, high)

Multiplying the 2 frequencies together allows us to recover the frequency of any individual category.

We define a variable x, which we call our wanted selections in total. Initially, we constrain x < 1/2 * n * f(mean, high)

We make the assumption that when a student is given an admission by both institutions, they will accept the offer from the superior institution with probability p, and 1-p in the other case.

We also make the assumption that for the institution that has p >= 1/2, they cannot distinguish between Q1 and Q2, so their offers in these respective categories break down wrt f(std, high). 

### Optimal policy for superior institution.
Since x < 1/2 * n * f(mean,high), the superior institution breaks its admissions at random into Q1 and Q2.

We keep track of the number of admissions handed out per category with a_i({OD, UD}). This also represents the strategy each player will be using in the "game" of admissions.

So, the naive and blind strategy for the OD after 0 best responses is:
a_1(OD) = x * f(std, high)
a_2(OD) = x * (1 - f(std, high))
a_3(OD) = 0
a_4(OD) = 0

### BR general form:
The underdog knows what the overdog institution would do in the naive case, so they adjust their strategy. Below we describe the algorithm to determine the best response strategy.

if x > 0 (we can still give out more admissions)
for best_category in categories:
    if x > d_(best_category) - a_(best_category):
        set a_(best_category) = d_(best_category)
        update x by calculating the expected number of students attending using a_(best)(OD) and a_(best)(UD). Subtract the increase in students
    else:
        calculate how many to accept for the final category

In this algorithm, we have to calculate how many we can accept. We can construct this backwards from our equations. First, let us see how to calculate the number of collisions/number of expected students.

### Calculating attendees
We can calculate the number of attendees y = selected(1*(1-a_i(OD)/d_i + p*(a_i(OD)/d_i))). Given y, we can calculate this value.
We also use this equation to calculate by how much we should reduce our expected number of incoming students.



In [2]:
# specifying environment variables
n = 100 # the number of applicants
frequency_dictionary = {
    "f_sigma" : 1/4,
    "f_mu" : 2/5
}
f_sigma = frequency_dictionary["f_sigma"] # the percentage of students that are unconvetionally talented
f_mu = frequency_dictionary["f_mu"] # the percentage of students that are conventionally talented


x = 1/3 * f_mu * n # the amount of students we're seeking to admit
p = 1/3 # probability of underdog winning competition
size_dictionary = {}


# specify the sizes of each section
def set_dict_size():
    global size_dictionary
    size_dictionary = {
        "Q1":f_sigma * f_mu * n,
        "Q2":(1-f_sigma) * f_mu * n,
        "Q3":f_sigma * (1-f_mu) * n,
        "Q4":(1-f_sigma) * (1-f_mu) * n
    }

def set_feq(mu_high, sigma_high):
    global frequency_dictionary
    frequency_dictionary = {
        "f_sigma" : 1/4,
        "f_mu" : 2/5
    }
    f_sigma = frequency_dictionary["f_sigma"] # the percentage of students that are unconvetionally talented
    f_mu = frequency_dictionary["f_mu"] # the percentage of students that are conventionally talented
    set_dict_size()


strategy_dictionary = {}


def reset_strategies():
    # specify the agent strategies
    global strategy_dictionary
    strategy_dictionary = {
    "overdog":{
        "Q1":0,
        "Q2":0,
        "Q3":0,
        "Q4":0
    }, 
    "underdog":{
        "Q1":0,
        "Q2":0,
        "Q3":0,
        "Q4":0
    }}

reset_strategies()

# specify the parameters from distribution sampling
distro_params = {
    "mu_high" : 5,
    "mu_low" : 4,
    "sigma_high" : 2,
    "sigma_low" : 1
}

cat_to_distro = {
    "Q1" : [distro_params["mu_high"]+1/2,distro_params["sigma_high"]],
    "Q2" : [distro_params["mu_high"], distro_params["sigma_low"]],
    "Q3" : [distro_params["mu_low"]+1/2,distro_params["sigma_high"]],
    "Q4" : [distro_params["mu_low"], distro_params["sigma_low"]],
}
print(size_dictionary)
print(x, f_sigma)
print(strategy_dictionary)


{'Q1': 10.0, 'Q2': 30.000000000000004, 'Q3': 15.0, 'Q4': 44.99999999999999}
13.333333333333334 0.25
{'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}


In [3]:
# In this cell we calculate the iterated best responses

def calc_collisions(category, strategies:dict)->float:
    numSelect = strategies["overdog"][category]
    numSelect2 = strategies["underdog"][category]
    collisions = numSelect * numSelect2 / size_dictionary[category]
    #print("num collisions", collisions)
    return collisions

# first we specify our function for calculating how much we should select
def calcSelectionNotBlind(allowed, strategies:dict, category_size:int, category:str, player:str, opposition:str):
    if allowed == 0 or allowed == float(0):
        return 0
    if player == "overdog":
        q = 1-p
    else:
        q = p
    opposition_selection_prob = strategies[opposition][category]/size_dictionary[category]
    intial_selection_number = strategies[player][category]
    if allowed > category_size:
        strategies[player][category] = category_size
    else:
        # in the case its less, use the formula specified
        to_add = allowed/(1-opposition_selection_prob+q*opposition_selection_prob)
        #print("want to admit", to_add)
        if to_add > category_size: # in the case we would go over (possible if selection frequency is high) cap it
            strategies[player][category] = category_size
        else:
            strategies[player][category] = to_add

    return allowed - (strategies[player][category] - calc_collisions(category, strategies))




def find_best_responseNotBlind(allowed, strategies:dict, player:str, opposition:str):
    # in this function we loop through all categories in order to find the best response

    for cat in size_dictionary.keys():
        #print(size_dictionary[cat])
        allowed = calcSelectionNotBlind(allowed, strategies, size_dictionary[cat], cat, player, opposition)
        #print()
        #print("allowed selections remaining", allowed)


print("before",strategy_dictionary)
#find_best_responseNotBlind(x, strategy_dictionary, "underdog", "overdog")
print("after",strategy_dictionary)


before {'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}
after {'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}


In [4]:
def calcSelectionBlind(allowed, strategies:dict, player:str, opposition:str, mu="high"):
    if allowed == 0 or allowed == float(0):
        return 0
    
    if mu == "high":
        # merge the upper two categories to get our needed category
        category_size = size_dictionary["Q1"] + size_dictionary["Q2"]
        opposition_selection_prob = (strategies[opposition]["Q1"] + strategies[opposition]["Q2"])/(category_size)
    else:
        category_size = size_dictionary["Q3"] + size_dictionary["Q4"]
        opposition_selection_prob = (strategies[opposition]["Q3"] + strategies[opposition]["Q4"])/(category_size)

    if allowed > category_size:
        breakdownBlindSelection(category_size, mu, strategies, player)
    else:
        # in the case its less, use the formula specified
        to_add = allowed/(1-opposition_selection_prob+(1-p)*opposition_selection_prob)
        #print("want to admit", to_add)
        if to_add > category_size: # in the case we would go over (possible if selection frequency is high) cap it
            breakdownBlindSelection(category_size, mu, strategies, player)
        else:
            breakdownBlindSelection(to_add, mu, strategies, player)

    if mu == "high":
        expected1 = strategies[player]["Q1"] - calc_collisions("Q1", strategies)
        expected2 = strategies[player]["Q2"] - calc_collisions("Q2", strategies)
    else:
        expected1 = strategies[player]["Q3"] - calc_collisions("Q3", strategies)
        expected2 = strategies[player]["Q4"] - calc_collisions("Q4", strategies)
    return allowed - expected1 - expected2

def breakdownBlindSelection(number, mu, strategies, player):
    num1 = number * f_sigma
    num2 = number * (1-f_sigma)
    if mu == "high":
        strategies[player]["Q1"] = num1
        strategies[player]["Q2"] = num2
    else:
        strategies[player]["Q3"] = num1
        strategies[player]["Q4"] = num2

def find_best_responseBlind(allowed, strategies:dict, player:str, opposition:str):
    # in this function we loop through all categories in order to find the best response
    mus = ["high", "low"]
    for mu in mus:
        allowed = calcSelectionBlind(allowed, strategies, player, opposition, mu)
        #print()
        #print("allowed selections remaining", allowed)


In [5]:
#find_best_responseBlind(x, strategy_dictionary, "overdog", "underdog")

print(strategy_dictionary)

{'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}


## Find equilibria
Keep iterating until strategies don't change

In [6]:


def copy_strategy(strategies, player):
    return list(strategies[player].values())

def find_strategies(p_val:float, blind=False, naive=10000, reverse=False):
    reset_strategies()
    change = 100000
    global p
    p = p_val
    iterations = 0
    while change > 0.001 or iterations < naive:
        strat_vals = copy_strategy(strategy_dictionary, "overdog")

        #print(strat_vals) # print strategy values
        if not reverse:
            find_best_responseNotBlind(x, strategy_dictionary, "underdog", "overdog")
            if blind:
                find_best_responseBlind(x, strategy_dictionary, "overdog", "underdog")
            else:
                find_best_responseNotBlind(x, strategy_dictionary, "overdog", "underdog")

        else:
            if blind:
                find_best_responseBlind(x, strategy_dictionary, "overdog", "underdog")
            else:
                find_best_responseNotBlind(x, strategy_dictionary, "overdog", "underdog")
            find_best_responseNotBlind(x, strategy_dictionary, "underdog", "overdog")

        updated = copy_strategy(strategy_dictionary, "overdog")


        ''' Debugging information
        print(strat_vals)
        print(updated)
        totalCollisions = sum([calc_collisions(key, strategy_dictionary) for key in cat_to_distro.keys()])
        print("Expected number of collisions is", totalCollisions)
        print("Total attempted admissions is", sum(updated))
        print("Admissions - collisions",  sum(updated) - totalCollisions)
        print(x)
        if iterations == 0:
            #print(strategy_dictionary["overdog"])

        '''

        result = [a_i - b_i for a_i, b_i in zip(updated, strat_vals)]
        change = max(result)
        iterations += 1
    #print()
    #print("converged to equilibrium in", iterations, "iterations")
    print(strategy_dictionary)


find_strategies(p, frequency_dictionary, False)
find_strategies(p, frequency_dictionary, blind=False, reverse=True)
#print()
#print("converged to equilibrium in", iterations, "iterations")
#print(strategy_dictionary)



{'overdog': {'Q1': 10.0, 'Q2': 17.625949659921606, 'Q3': 10.444800364889293, 'Q4': 3.77420139763227}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856194117049, 'Q3': 8.011694250123705, 'Q4': 1.9696634425970672}}
{'overdog': {'Q1': 10.0, 'Q2': 17.625946430792162, 'Q3': 10.444738146683632, 'Q4': 3.7740849867059887}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856600569088, 'Q3': 8.011760237082896, 'Q4': 1.9697026967651854}}


## Play the game
We implement the actual game result, to see how much advantage can be gained

In [7]:
def get_proportion(result):
    ''' returns the proportion of the underdog achieved'''
    return result["under"]/(result["over"]+result["under"])
def play_game(strategies:dict):
    '''This function takes the strategies of both players in the form of a dictionary and simulates a game playing out. Outputting a dictionary with the same player names and the returned achieved value.
    Input:
        strategies : dict
    Output:
        results : dict'''
    
    # for each player calculate the number of rolls they have in each category and simulate them - round to the nearest whole number. We then take the average of the result

    over_strats = strategies["overdog"]
    under_strats = strategies["underdog"]

    
    over = []
    under = []
    for key in over_strats.keys():
        wantedRollsOver = over_strats[key]
        wantedRollsUnder = under_strats[key]
        collisions = calc_collisions(key, strategies)
        overRolls = wantedRollsOver - collisions*(p) # have to invert these since we're subtracting the number of cases in which we don't win
        underRolls = wantedRollsUnder - collisions*(1-p)

        mu, sigma = cat_to_distro[key][0], cat_to_distro[key][1]
        over = np.concatenate((over, np.random.normal(loc=mu, scale=sigma, size=round(overRolls))))
        under = np.concatenate((under,np.random.normal(loc=mu, scale=sigma, size=round(underRolls))))
    
    #print(over)
    #print(under)
    result = {
        "over" : np.mean(over),
        "under" : np.mean(under)
    }

    return result
proportions = []
for i in range(100):
    find_strategies(p,frequency_dictionary, False)
    result = play_game(strategy_dictionary)
    proportions.append(get_proportion(result))
print(result)

print(np.mean(proportions), np.std(proportions))


{'overdog': {'Q1': 10.0, 'Q2': 17.625949659921606, 'Q3': 10.444800364889293, 'Q4': 3.77420139763227}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856194117049, 'Q3': 8.011694250123705, 'Q4': 1.9696634425970672}}
{'overdog': {'Q1': 10.0, 'Q2': 17.625949659921606, 'Q3': 10.444800364889293, 'Q4': 3.77420139763227}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856194117049, 'Q3': 8.011694250123705, 'Q4': 1.9696634425970672}}
{'overdog': {'Q1': 10.0, 'Q2': 17.625949659921606, 'Q3': 10.444800364889293, 'Q4': 3.77420139763227}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856194117049, 'Q3': 8.011694250123705, 'Q4': 1.9696634425970672}}
{'overdog': {'Q1': 10.0, 'Q2': 17.625949659921606, 'Q3': 10.444800364889293, 'Q4': 3.77420139763227}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856194117049, 'Q3': 8.011694250123705, 'Q4': 1.9696634425970672}}
{'overdog': {'Q1': 10.0, 'Q2': 17.625949659921606, 'Q3': 10.444800364889293, 'Q4': 3.77420139763227}, 'underdog': {'Q1': 10.0, 'Q2': 21.91856194117049, 'Q3': 8.011694250123705, 'Q4': 1

### Graphing variables
We're going to range the overdog/underdog dynamic, and also vary the distributions of the talent pool, as well as how naive/blind the overdog is